In [21]:
import json  
from csv import reader
import pandas as pd
import numpy as np
import math
import io

rating_file = '../data/job_user_rating.csv'
jobs_file = '../data/all_jobs.json'
users_file = '../data/all_users.json'
train_file = '../data/job_user_train.txt'
test_file = '../data/job_user_test.txt'
validation_file = '../data/job_user_validation.txt'

with open(jobs_file, "r+") as infile:
    jobs = json.load(infile) #Read json file into buffer

try:
    rating_df = pd.read_csv(rating_file, header=0)
    print("Loading past rating records...")
    number_of_jobs, number_of_users = rating_df.shape
except OSError as err:
    print("OS Error:", err)
    print(f"No file named", rating_file)
    raise 

with open(jobs_file, 'r') as jobs:
    job_obj = json.load(jobs)
# print(len(job_obj))
res = list(filter(lambda x:x["Job_ID"]=="31", job_obj))[0]
# print(f'jid 31 is:', res)
# print(f'desc of jid 31 is:', res['Short_Description'])
# print(type(res))

with open(users_file, 'r') as users:
    user_obj = json.load(users)

# print(user_obj)
ures = list(filter(lambda x:x["User_ID"]=="12", user_obj))[0]
# print(f'user exp: ', ures['experience'])

Loading past rating records...


In [22]:
valid_rating_lookup_table = {}
for uid, rating_srs in rating_df.iteritems():
    counter = 0
    for jid, rating in rating_srs.iteritems():
        if not math.isnan(rating):
            valid_rating_lookup_table[(uid,str(jid))] = rating
            counter += 1
#     print(f'uid: ', uid, 'has', counter, 'ratings \n')
# print(valid_rating_lookup_table)
print(len(valid_rating_lookup_table))


942


C:\Users\jbc-zeph21\AppData\Local\Temp\ipykernel_28260\1019653562.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for uid, rating_srs in rating_df.iteritems():
C:\Users\jbc-zeph21\AppData\Local\Temp\ipykernel_28260\1019653562.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for jid, rating in rating_srs.iteritems():


In [23]:
# Prepare dataframe for output
row_list = []
for (row,((uid,jid), rating)) in enumerate(valid_rating_lookup_table.items()):
    rel_score = rating
    user = list(filter(lambda x:x["User_ID"]==uid, user_obj))[0]
    job = list(filter(lambda x:x["Job_ID"]==jid, job_obj))[0]
    job_url = job['Job_Detail_Link']
    job_title = job['Job_Title']
    query_exp = user['experience']
    job_desc = job['Short_Description']
    row_list.append([rel_score,job_url,job_title,query_exp,job_desc])
df_output = pd.DataFrame(row_list)
df_output.columns = ['rel_score', 'job_url', 'job_title', 'query_exp', 'job_desc']
print(df_output.head(5))
print(len(df_output))



   rel_score                                            job_url  \
0        1.0  https://www.indeed.com/company/quinnox/jobs/Da...   
1        4.0  https://www.indeed.com/rc/clk?jk=aeb5f5aaac099...   
2        3.0  https://www.indeed.com/rc/clk?jk=920cc85ba0d3a...   
3        5.0  https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...   
4        1.0  https://www.indeed.com/rc/clk?jk=e56cd4fb3a53f...   

                                           job_title  \
0                                     Data Scientist   
1                                     Data Scientist   
2                   Associate Director, Data Science   
3                              Manager, Data Science   
4  Marketing Senior Manager - Strategic Marketing...   

                                           query_exp  \
0  [Software Developer - Intern, ABC Corporation ...   
1  [Software Developer - Intern, ABC Corporation ...   
2  [Software Developer - Intern, ABC Corporation ...   
3  [Software Developer - Intern, ABC

In [26]:
# Split the valid data into training and testing sets and write output files
df_train = df_output.sample(frac = 0.7)
df_test = df_output.drop(df_train.index).sample(frac = 0.5)
df_validation = df_output.drop(df_train.index).drop(df_test.index)
print(f'number of training samples: ', len(df_train))
print(f'number of testing samples: ', len(df_test))
print(f'number of validation samples: ', len(df_validation))

train_line_num = 0
with io.open(train_file, 'w', encoding='utf-8') as file_output:
    print(len(df_train))
    # line = ''
    # for i in range(len(df_train)):
    #     line = str(df_train.iloc[i][0]) + '<SPLIT>' + \
    #         str(df_train.iloc[i][1]) + '<SPLIT>' + \
    #         str(df_train.iloc[i][2]) + '<SPLIT>' + \
    #         str(df_train.iloc[i][3]) + '<SPLIT>' + \
    #         str(df_train.iloc[i][4])
    #     file_output.write(line)
    #     file_output.write('\n')
    #     train_line_num += 1
print(f'train line number is: ', train_line_num)

test_line_num = 0
with open(test_file, 'w') as file_output:
    line = ''
    for i in range(len(df_test)):
        line = str(df_test.iloc[i][0]) + '<SPLIT>' + \
            str(df_test.iloc[i][1]) + '<SPLIT>' + \
            str(df_test.iloc[i][2]) + '<SPLIT>' + \
            str(df_test.iloc[i][3]) + '<SPLIT>' + \
            str(df_test.iloc[i][4])
        file_output.write(line)
        file_output.write('\n')
        test_line_num += 1

print(f'testline number is: ', test_line_num)

validation_line_num = 0
with open(validation_file, 'w') as file_output:
    line = ''
    for i in range(len(df_test)):
        line = str(df_test.iloc[i][0]) + '<SPLIT>' + \
            str(df_test.iloc[i][1]) + '<SPLIT>' + \
            str(df_test.iloc[i][2]) + '<SPLIT>' + \
            str(df_test.iloc[i][3]) + '<SPLIT>' + \
            str(df_test.iloc[i][4])
        file_output.write(line)
        file_output.write('\n')
        validation_line_num += 1

print(f'validation number is: ', validation_line_num)

number of training samples:  659
number of testing samples:  142
number of validation samples:  141


TypeError: 'str' object cannot be interpreted as an integer